In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

In [7]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  306534 non-null  object
 1   event        306534 non-null  object
 2   value        306534 non-null  object
 3   time         306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [2]:
cust = pd.read_csv('customers.csv')
offers = pd.read_csv('offers.csv')
events = pd.read_csv('events.csv')

#Converting the events value column from a string into a column of dictionaries for explosion
events['value'] = events.value.apply(eval)

#Editing the 'became_member_on' field for proper converstion to datetime
cust['became_member_on'] = pd.to_datetime(cust['became_member_on'],format='%Y%m%d')

#Declaring bins for demographics agg purposes
age_bins= [0,18,35,55,75,110]
age_labels = ['0 to 17','18 to 34','35 to 54','55 to 74','75 +']

spend_bins = [0,49,101,2000]
spend_labels = ['Standard','Loyal','Devoted']

inc_bins = [0,30001,58021,94001,153001,99999999999]
inc_labels = ['Low','Lower-Mid','Middle','Upper-Middle','Upper']

member_age_bins = [0,1,2,3,4,5,6,7,8,9,10,11]
member_age_labels = ['1 year','2 years','3 years','4 years','5 years',
              '6 years','7 years','8 years','9 years','10 years','11 years']

In [13]:
#Original offers table data usage = 2.7KB (No Optimization needed - Impact negligible)

offers = (offers
    .assign(
        email_offer = offers['channels'].str.contains('email'),
        web_offer = offers['channels'].str.contains('web'),
        mobile_offer = offers['channels'].str.contains('mobile'),
        social_offer = offers['channels'].str.contains('social')))


#cust table data usage = Start: 2.7MB; Optimized: 1.6MB
cust = cust.astype({
    'became_member_on':'datetime64[ns]',
    'gender':'category',
    'age':'int16',
    'income':'category',
})

#Original events table data usage = 75.6MB; Optimized: 46.3
    #Note, this optimization also has the added benefit of exploding out the data in 'value'
events = (events
    .assign(
        trans=events['value'].apply(lambda x: x.get('amount', None)),
        offer_send=events['value'].apply(lambda x: x.get('offer id', None)),
        offer_redeem=events['value'].apply(lambda x: x.get('offer_id', None)),
        reward=events['value'].apply(lambda x: x.get('reward', None)),
        dt = (pd.to_datetime('2024-07-08 00:00:00') + (pd.to_timedelta(1,unit='H')*events['time'])))
    .astype({
        'event':'category',
        'reward':'float32',
        'trans':'float32'}))

KeyError: 'value'

In [14]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   dt           306534 non-null  datetime64[ns]
 1   event        306534 non-null  category      
 2   customer_id  306534 non-null  object        
 3   trans        138953 non-null  float32       
 4   reward       33579 non-null   float32       
 5   offer_id     306534 non-null  object        
dtypes: category(1), datetime64[ns](1), float32(2), object(2)
memory usage: 9.6+ MB


In [15]:
#Reording events table to be more readable and removingn duplicate columns caused by naming conventions in 'value'
events = events[['dt','event','customer_id','trans','reward','offer_send','offer_redeem']]
events['offer_id'] = events['offer_send'].fillna('') + events['offer_redeem'].fillna('')
events = events.drop(['offer_redeem','offer_send'],axis=1)

KeyError: "['offer_send', 'offer_redeem'] not in index"

In [16]:
#Creating individual tables for each promo's events.  This will allow easier time analysis
#Additionally, creating a dataframe with just transaction log

bogo_a = (events[events['offer_id'].str.contains("ae264e3637204a6fb9bb56bc8210ddfd")]
          .drop(['offer_id','trans'],axis=1))
bogo_b = (events[events['offer_id'].str.contains("4d5c57ea9a6940dd891ad53e9dbe8da0")]
          .drop(['offer_id','trans'],axis=1))
bogo_c = (events[events['offer_id'].str.contains("9b98b8c7a33c4b65b9aebfe6a799e6d9")]
          .drop(['offer_id','trans'],axis=1))
bogo_d = (events[events['offer_id'].str.contains("f19421c1d4aa40978ebb69ca19b0e20d")]
          .drop(['offer_id','trans'],axis=1))
info_a = (events[events['offer_id'].str.contains("3f207df678b143eea3cee63160fa8bed")]
          .drop(['offer_id','trans'],axis=1))
info_b = (events[events['offer_id'].str.contains("5a8bc65990b245e5a138643cd4eb9837")]
          .drop(['offer_id','trans'],axis=1))
disc_a = (events[events['offer_id'].str.contains("0b1e1539f2cc45b7b9fa7c272da2e1d7")]
          .drop(['offer_id','trans'],axis=1))
disc_b = (events[events['offer_id'].str.contains("2298d6c36e964ae4a3e7e9706d1fb8c2")]
          .drop(['offer_id','trans'],axis=1))
disc_c = (events[events['offer_id'].str.contains("fafdcd668e3743c1bb461111dcafc2a4")]
          .drop(['offer_id','trans'],axis=1))
disc_d = (events[events['offer_id'].str.contains("2906b810c7d4411798c6938adc9daaa5")]
          .drop(['offer_id','trans'],axis=1))
info_a = (events[events['offer_id'].str.contains("3f207df678b143eea3cee63160fa8bed")]
          .drop(['offer_id','trans'],axis=1))
info_b = (events[events['offer_id'].str.contains("5a8bc65990b245e5a138643cd4eb9837")]
          .drop(['offer_id','trans'],axis=1))


trans_log = (events[events['event'].str.contains("transaction")]
            .drop(['reward','offer_id','event'],axis=1))

In [17]:
cust_reca = (info_a
     [info_a['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_viewa = (info_a
     [info_a['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_recb = (info_b
     [info_b['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_viewb = (info_b
     [info_b['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

info_a = (cust_reca
    .merge(cust_viewa,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed'}))

info_b = (cust_recb
    .merge(cust_viewb,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed'}))

In [18]:
#Excluding all cases where we are unsure if/when customer viewed communication
info_a = info_a[info_a['offer_viewed'].notnull()]
info_b = info_b[info_b['offer_viewed'].notnull()]

In [19]:
#Creating a list of transactions that is only customers in the info_a/info_b lists above
info_a_trans = trans_log[trans_log['customer_id'].isin(info_a['customer_id'])]
info_b_trans = trans_log[trans_log['customer_id'].isin(info_b['customer_id'])]

#Merging with info_trans in many to one to create the ability to 
#analyze in-row for transactions prior to and after viewing
info_a = (info_a_trans
    .merge(info_a,how='left',validate='m:1'))
info_b = (info_b_trans
    .merge(info_b,how='left',validate='m:1'))


In [20]:
#Adding a column to info dataframes that will calculate transactions based on the epoch (before and after)
#of the customer viewing the transaction.

info_a['hrdiff'] = ((info_a['dt']) - info_a['offer_viewed']) / pd.Timedelta(hours=1)
info_b['hrdiff'] = ((info_b['dt']) - info_b['offer_viewed']) / pd.Timedelta(hours=1)

In [21]:
#Creating a df to hold purchase details for segmentation
purch_agg = (trans_log
     .groupby(['customer_id'])['trans']
     .agg(['sum', 'mean','count'])
     .round(0)
     .rename(columns=
            {'sum':'ttl_spend','mean':'avg_spend','count':'visit_cnt'}
    ))
purch_agg.reset_index(inplace=True)

#Using that dataframe to create a customer profiling dataframe
cust_profiling = purch_agg.merge(
    cust,how='left')
cust_profiling = cust_profiling.assign(
         member_age = ((pd.to_datetime('2024-07-08 00:00:00') - cust_profiling['became_member_on']) / pd.Timedelta(days=365.25)),
         age_group = pd.cut(cust_profiling['age'], bins=age_bins, labels=age_labels, right=False),
         spend_level = pd.cut(cust_profiling['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         inc_level = pd.cut(cust_profiling['income'], bins=inc_bins, labels=inc_labels, right=False))

In [22]:
#Shame on me for being too lazy to write a lambda function in the last cell
cust_profiling = cust_profiling.assign(
                    member_age_grp = pd.cut(cust_profiling['member_age'], 
                        bins=member_age_bins, labels=member_age_labels, right=False))

In [23]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #bogo_a
cust_rec = (bogo_a
     [bogo_a['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (bogo_a
     [bogo_a['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (bogo_a
     [bogo_a['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

bogo_a = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'}))

#Adding calculated columns:

bogo_a = (bogo_a
     .assign(
         age_group = pd.cut(bogo_a['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(bogo_a['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (bogo_a['offer_viewed'] - bogo_a['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (bogo_a['offer_redeemed'] - bogo_a['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (bogo_a['offer_redeemed'] - bogo_a['offer_rcvd']) / pd.Timedelta(hours=1)))

In [24]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #bogo_b
cust_rec = (bogo_b
     [bogo_b['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (bogo_b
     [bogo_b['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (bogo_b
     [bogo_b['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

bogo_b = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'
    }))

#Adding calculated columns:

bogo_b = (bogo_b
     .assign(
         age_group = pd.cut(bogo_b['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(bogo_b['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (bogo_b['offer_viewed'] - bogo_b['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (bogo_b['offer_redeemed'] - bogo_b['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (bogo_b['offer_redeemed'] - bogo_b['offer_rcvd']) / pd.Timedelta(hours=1)))

In [25]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #bogo_c
cust_rec = (bogo_c
     [bogo_c['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (bogo_c
     [bogo_c['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (bogo_c
     [bogo_c['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

bogo_c = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'
    }))

#Adding calculated columns:

bogo_c = (bogo_c
     .assign(
         age_group = pd.cut(bogo_c['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(bogo_c['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (bogo_c['offer_viewed'] - bogo_c['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (bogo_c['offer_redeemed'] - bogo_c['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (bogo_c['offer_redeemed'] - bogo_c['offer_rcvd']) / pd.Timedelta(hours=1)))

In [26]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #bogo_d
cust_rec = (bogo_d
     [bogo_d['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (bogo_d
     [bogo_d['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (bogo_d
     [bogo_d['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

bogo_d = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'
    }))

#Adding calculated columns:

bogo_d = (bogo_d
     .assign(
         age_group = pd.cut(bogo_d['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(bogo_d['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (bogo_d['offer_viewed'] - bogo_d['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (bogo_d['offer_redeemed'] - bogo_d['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (bogo_d['offer_redeemed'] - bogo_d['offer_rcvd']) / pd.Timedelta(hours=1)))

In [27]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #disc_a
cust_rec = (disc_a
     [disc_a['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (disc_a
     [disc_a['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (disc_a
     [disc_a['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

disc_a = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'
    }))

#Adding calculated columns:

disc_a = (disc_a
     .assign(
         age_group = pd.cut(disc_a['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(disc_a['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (disc_a['offer_viewed'] - disc_a['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (disc_a['offer_redeemed'] - disc_a['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (disc_a['offer_redeemed'] - disc_a['offer_rcvd']) / pd.Timedelta(hours=1)))

In [28]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #disc_b
cust_rec = (disc_b
     [disc_b['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (disc_b
     [disc_b['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (disc_b
     [disc_b['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

disc_b = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'
    }))

#Adding calculated columns:

disc_b = (disc_b
     .assign(
         age_group = pd.cut(disc_b['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(disc_b['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (disc_b['offer_viewed'] - disc_b['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (disc_b['offer_redeemed'] - disc_b['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (disc_b['offer_redeemed'] - disc_b['offer_rcvd']) / pd.Timedelta(hours=1)))

In [29]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #disc_c
cust_rec = (disc_c
     [disc_c['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (disc_c
     [disc_c['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (disc_c
     [disc_c['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

disc_c = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'
    }))

#Adding calculated columns:

disc_c = (disc_c
     .assign(
         age_group = pd.cut(disc_c['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(disc_c['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (disc_c['offer_viewed'] - disc_c['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (disc_c['offer_redeemed'] - disc_c['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (disc_c['offer_redeemed'] - disc_c['offer_rcvd']) / pd.Timedelta(hours=1)))

In [30]:
#Rebuilding offer records to be unique lines with first receipt, first view, and redemption datetimes
    #disc_d
cust_rec = (disc_d
     [disc_d['event'].str.contains('offer received')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'rcvd'}))

cust_view = (disc_d
     [disc_d['event'].str.contains('offer viewed')]
     .groupby(['customer_id'])['dt']
     .min()
     .reset_index()
     .rename(columns={'dt':'view'}))

cust_redeem = (disc_d
     [disc_d['event'].str.contains('offer completed')]
     .groupby(['customer_id'])[['dt','reward']]
     .min()
     .reset_index()
     .rename(columns={'dt':'redeem'}))

disc_d = (cust_rec
    .merge(cust_view,how='left')
    .merge(cust_redeem,how='left')
    .merge(cust,how='left')
    .merge(purch_agg,how='left')
    .rename(columns={
        'rcvd':'offer_rcvd',
        'view':'offer_viewed',
        'redeem':'offer_redeemed'
    }))

#Adding calculated columns:

disc_d = (disc_d
     .assign(
         age_group = pd.cut(disc_d['age'], bins=age_bins, labels=age_labels, right=False),
         cust_level = pd.cut(disc_d['ttl_spend'], bins=spend_bins, labels=spend_labels, right=False),
         hrs_rcvd_view = (disc_d['offer_viewed'] - disc_d['offer_rcvd']) / pd.Timedelta(hours=1),
         hrs_view_redeem = (disc_d['offer_redeemed'] - disc_d['offer_viewed']) / pd.Timedelta(hours=1),
         hrs_rcvd_redeem = (disc_d['offer_redeemed'] - disc_d['offer_rcvd']) / pd.Timedelta(hours=1)))

In [31]:
#Creating a list of transactions that is only customers in the bogo lists above
bogo_a_trans = trans_log[trans_log['customer_id'].isin(bogo_a['customer_id'])]
bogo_b_trans = trans_log[trans_log['customer_id'].isin(bogo_b['customer_id'])]
bogo_c_trans = trans_log[trans_log['customer_id'].isin(bogo_c['customer_id'])]
bogo_d_trans = trans_log[trans_log['customer_id'].isin(bogo_d['customer_id'])]

disc_a_trans = trans_log[trans_log['customer_id'].isin(disc_a['customer_id'])]
disc_b_trans = trans_log[trans_log['customer_id'].isin(disc_b['customer_id'])]
disc_c_trans = trans_log[trans_log['customer_id'].isin(disc_c['customer_id'])]
disc_d_trans = trans_log[trans_log['customer_id'].isin(disc_d['customer_id'])]

#Merging with above transaction lists in many to one to create the ability to 
#analyze in-row for transactions prior to and after viewing
bogo_a_trans = (bogo_a_trans
    .merge(bogo_a,how='left',validate='m:1'))
bogo_b_trans = (bogo_b_trans
    .merge(bogo_b,how='left',validate='m:1'))
bogo_c_trans = (bogo_c_trans
    .merge(bogo_c,how='left',validate='m:1'))
bogo_d_trans = (bogo_d_trans
    .merge(bogo_d,how='left',validate='m:1'))

disc_a_trans = (disc_a_trans
    .merge(disc_a,how='left',validate='m:1'))
disc_b_trans = (disc_b_trans
    .merge(disc_b,how='left',validate='m:1'))
disc_c_trans = (disc_c_trans
    .merge(disc_c,how='left',validate='m:1'))
disc_d_trans = (disc_d_trans
    .merge(disc_d,how='left',validate='m:1'))

In [32]:
#Removing all entries from this ledger where the customer did not view the promotion

bogo_a_trans = (bogo_a_trans[bogo_a_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])
bogo_b_trans = (bogo_b_trans[bogo_b_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])
bogo_c_trans = (bogo_c_trans[bogo_c_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])
bogo_d_trans = (bogo_d_trans[bogo_d_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])

disc_a_trans = (disc_a_trans[disc_a_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])
disc_b_trans = (disc_b_trans[disc_b_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])
disc_c_trans = (disc_c_trans[disc_c_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])
disc_d_trans = (disc_d_trans[disc_d_trans['hrs_rcvd_view'].notnull()]
                [['dt','customer_id','trans','offer_rcvd','offer_viewed']])

In [33]:
#Adding a column to dataframes that will calculate transactions based on the epoch (before and after)
#of the customer viewing the transaction.

bogo_a_trans['hrdiff'] = ((bogo_a_trans['dt']) - bogo_a_trans['offer_viewed']) / pd.Timedelta(hours=1)
bogo_b_trans['hrdiff'] = ((bogo_b_trans['dt']) - bogo_b_trans['offer_viewed']) / pd.Timedelta(hours=1)
bogo_c_trans['hrdiff'] = ((bogo_c_trans['dt']) - bogo_c_trans['offer_viewed']) / pd.Timedelta(hours=1)
bogo_d_trans['hrdiff'] = ((bogo_d_trans['dt']) - bogo_d_trans['offer_viewed']) / pd.Timedelta(hours=1)

disc_a_trans['hrdiff'] = ((disc_a_trans['dt']) - disc_a_trans['offer_viewed']) / pd.Timedelta(hours=1)
disc_b_trans['hrdiff'] = ((disc_b_trans['dt']) - disc_b_trans['offer_viewed']) / pd.Timedelta(hours=1)
disc_c_trans['hrdiff'] = ((disc_c_trans['dt']) - disc_c_trans['offer_viewed']) / pd.Timedelta(hours=1)
disc_d_trans['hrdiff'] = ((disc_d_trans['dt']) - disc_d_trans['offer_viewed']) / pd.Timedelta(hours=1)

In [34]:
#Bogo promotion summary and segmentation tables

#Offers Provided
bogoa_rec = bogo_a['customer_id'].nunique()
bogob_rec = bogo_b['customer_id'].nunique()
bogoc_rec = bogo_c['customer_id'].nunique()
bogod_rec = bogo_d['customer_id'].nunique()

#Offers Viewed
bogoa_view = bogo_a['hrs_rcvd_view'].notna().sum()
bogob_view = bogo_b['hrs_rcvd_view'].notna().sum()
bogoc_view = bogo_c['hrs_rcvd_view'].notna().sum()
bogod_view = bogo_d['hrs_rcvd_view'].notna().sum()

#Offers Redeemed
bogoa_redcnt = bogo_a['hrs_rcvd_redeem'].notna().sum()
bogob_redcnt = bogo_b['hrs_rcvd_redeem'].notna().sum()
bogoc_redcnt = bogo_c['hrs_rcvd_redeem'].notna().sum()
bogod_redcnt = bogo_d['hrs_rcvd_redeem'].notna().sum()

#Redemption Percent
bogoa_redperc = ((bogo_a['hrs_rcvd_redeem'].notna().sum() / bogo_a['customer_id'].nunique()) * 100)
bogob_redperc = ((bogo_b['hrs_rcvd_redeem'].notna().sum() / bogo_b['customer_id'].nunique()) * 100)
bogoc_redperc = ((bogo_c['hrs_rcvd_redeem'].notna().sum() / bogo_c['customer_id'].nunique()) * 100)
bogod_redperc = ((bogo_d['hrs_rcvd_redeem'].notna().sum() / bogo_d['customer_id'].nunique()) * 100)

#24-Hour Redemption Percent
bogoa_redperc_24 = ((((bogo_a['hrs_rcvd_redeem'] < 25) * 1).sum() / bogo_a['hrs_rcvd_redeem'].notna().sum()) * 100)
bogob_redperc_24 = ((((bogo_b['hrs_rcvd_redeem'] < 25) * 1).sum() / bogo_b['hrs_rcvd_redeem'].notna().sum()) * 100)
bogoc_redperc_24 = ((((bogo_c['hrs_rcvd_redeem'] < 25) * 1).sum() / bogo_c['hrs_rcvd_redeem'].notna().sum()) * 100)
bogod_redperc_24 = ((((bogo_d['hrs_rcvd_redeem'] < 25) * 1).sum() / bogo_d['hrs_rcvd_redeem'].notna().sum()) * 100)

#Average Hours to View
bogoa_rcpt = bogo_a['hrs_rcvd_view'].mean().round(2)
bogob_rcpt = bogo_b['hrs_rcvd_view'].mean().round(2)
bogoc_rcpt = bogo_c['hrs_rcvd_view'].mean().round(2)
bogod_rcpt = bogo_d['hrs_rcvd_view'].mean().round(2)

#Percentage Redemptions 35 to 54
bogoa_3554 = ((bogo_a
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_a
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)
bogob_3554 = ((bogo_b
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_b
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)
bogoc_3554 = ((bogo_c
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_c
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)
bogod_3554 = ((bogo_d
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_d
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)

#Percentage Redemptions 55 to 74
bogoa_5574 = ((bogo_a
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_a
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)
bogob_5574 = ((bogo_b
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_b
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)
bogoc_5574 = ((bogo_c
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_c
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)
bogod_5574 = ((bogo_d
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    bogo_d
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)

#Percentage Standard Customer Redemptions
bogoa_stanperc = ((bogo_a
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_a
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)
bogob_stanperc = ((bogo_b
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_b
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)
bogoc_stanperc = ((bogo_c
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_c
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)
bogod_stanperc = ((bogo_d
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_d
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)

#Percentage Loyal Customer Redemptions
bogoa_loyperc = ((bogo_a
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_a
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)
bogob_loyperc = ((bogo_b
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_b
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)
bogoc_loyperc = ((bogo_c
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_c
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)
bogod_loyperc = ((bogo_d
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_d
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)

#Percentage Devoted Customer Redemptions
bogoa_devperc = ((bogo_a
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_a
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)
bogob_devperc = ((bogo_b
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_b
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)
bogoc_devperc = ((bogo_c
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_c
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)
bogod_devperc = ((bogo_d
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    bogo_d
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)

bogoname = ['Bogo A','Bogo B','Bogo C','Bogo D']
bogo_rec = [bogoa_rec, bogob_rec, bogoc_rec, bogod_rec]
bogo_view = [bogoa_view, bogob_view, bogoc_view, bogoa_redcnt]
bogo_redcnt = [bogoa_redcnt, bogob_redcnt, bogoc_redcnt, bogod_redcnt]
bogo_redperc = [bogoa_redperc, bogob_redperc, bogoc_redperc, bogod_redperc]
bogo_redperc_24 = [bogoa_redperc_24, bogob_redperc_24, bogoc_redperc_24, bogod_redperc_24]
bogo_rcpt = [bogoa_rcpt, bogob_rcpt, bogoc_rcpt, bogod_rcpt]
bogo_3554 = [bogoa_3554, bogob_3554, bogoc_3554, bogod_3554]
bogo_5574 = [bogoa_5574, bogob_5574, bogoc_5574, bogod_5574]
bogo_stanperc = [bogoa_stanperc, bogob_stanperc, bogoc_stanperc, bogod_stanperc]
bogo_loyperc = [bogoa_loyperc, bogob_loyperc, bogoc_loyperc, bogod_loyperc]
bogo_devperc = [bogoa_devperc, bogob_devperc, bogoc_devperc, bogod_devperc]


bogo_sum_data = ({
    'Campaign': bogoname,
    'Offers Provided': bogo_rec,
    'Offers Viewed': bogo_view,
    'Offers Redeemed': bogo_redcnt,
    'Average Hours to View': bogo_rcpt,
    'Redemption Percent': bogo_redperc,
    '24-Hour Redemption Percent': bogo_redperc_24})

bogo_seg_data = ({
    'Campaign': bogoname,
    'Percent 35-54 Redeemed': bogo_3554,
    'Percent 55-74 Redeemed': bogo_5574,
    'Standard Percent Redeemed': bogo_stanperc,
    'Loyal Percent Redeemed': bogo_loyperc,
    'Devoted Percent Redeemed': bogo_devperc})
    
bogo_summary = pd.DataFrame(bogo_sum_data).round(2)
bogo_segmentation = pd.DataFrame(bogo_seg_data).round(2)

In [35]:
#Disc promotion summary and segmentation tables

#Offers Provided
disca_rec = disc_a['customer_id'].nunique()
discb_rec = disc_b['customer_id'].nunique()
discc_rec = disc_c['customer_id'].nunique()
discd_rec = disc_d['customer_id'].nunique()

#Offers Viewed
disca_view = disc_a['hrs_rcvd_view'].notna().sum()
discb_view = disc_b['hrs_rcvd_view'].notna().sum()
discc_view = disc_c['hrs_rcvd_view'].notna().sum()
discd_view = disc_d['hrs_rcvd_view'].notna().sum()

#Offers Redeemed
disca_redcnt = disc_a['hrs_rcvd_redeem'].notna().sum()
discb_redcnt = disc_b['hrs_rcvd_redeem'].notna().sum()
discc_redcnt = disc_c['hrs_rcvd_redeem'].notna().sum()
discd_redcnt = disc_d['hrs_rcvd_redeem'].notna().sum()

#Redemption Percent
disca_redperc = ((disc_a['hrs_rcvd_redeem'].notna().sum() / disc_a['customer_id'].nunique()) * 100)
discb_redperc = ((disc_b['hrs_rcvd_redeem'].notna().sum() / disc_b['customer_id'].nunique()) * 100)
discc_redperc = ((disc_c['hrs_rcvd_redeem'].notna().sum() / disc_c['customer_id'].nunique()) * 100)
discd_redperc = ((disc_d['hrs_rcvd_redeem'].notna().sum() / disc_d['customer_id'].nunique()) * 100)

#24-Hour Redemption Percent
disca_redperc_24 = ((((disc_a['hrs_rcvd_redeem'] < 25) * 1).sum() / disc_a['hrs_rcvd_redeem'].notna().sum()) * 100)
discb_redperc_24 = ((((disc_b['hrs_rcvd_redeem'] < 25) * 1).sum() / disc_b['hrs_rcvd_redeem'].notna().sum()) * 100)
discc_redperc_24 = ((((disc_c['hrs_rcvd_redeem'] < 25) * 1).sum() / disc_c['hrs_rcvd_redeem'].notna().sum()) * 100)
discd_redperc_24 = ((((disc_d['hrs_rcvd_redeem'] < 25) * 1).sum() / disc_d['hrs_rcvd_redeem'].notna().sum()) * 100)

#Average Hours to View
disca_rcpt = disc_a['hrs_rcvd_view'].mean().round(2)
discb_rcpt = disc_b['hrs_rcvd_view'].mean().round(2)
discc_rcpt = disc_c['hrs_rcvd_view'].mean().round(2)
discd_rcpt = disc_d['hrs_rcvd_view'].mean().round(2)

#Percentage Redemptions 35 to 54
disca_3554 = ((disc_a
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_a
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)
discb_3554 = ((disc_b
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_b
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)
discc_3554 = ((disc_c
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_c
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)
discd_3554 = ((disc_d
     .query('age_group in ("35 to 54")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_d
     .query('age_group in ("35 to 54")')
     .loc[:, 'age_group'].count()) * 100)

#Percentage Redemptions 55 to 74
disca_5574 = ((disc_a
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_a
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)
discb_5574 = ((disc_b
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_b
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)
discc_5574 = ((disc_c
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_c
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)
discd_5574 = ((disc_d
     .query('age_group in ("55 to 74")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'age_group'].count() / 
    disc_d
     .query('age_group in ("55 to 74")')
     .loc[:, 'age_group'].count()) * 100)

#Percentage Standard Customer Redemptions
disca_stanperc = ((disc_a
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_a
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)
discb_stanperc = ((disc_b
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_b
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)
discc_stanperc = ((disc_c
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_c
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)
discd_stanperc = ((disc_d
     .query('cust_level in ("Standard")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_d
     .query('cust_level in ("Standard")')
     .loc[:, 'cust_level'].count()) * 100)

#Percentage Loyal Customer Redemptions
disca_loyperc = ((disc_a
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_a
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)
discb_loyperc = ((disc_b
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_b
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)
discc_loyperc = ((disc_c
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_c
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)
discd_loyperc = ((disc_d
     .query('cust_level in ("Loyal")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_d
     .query('cust_level in ("Loyal")')
     .loc[:, 'cust_level'].count()) * 100)

#Percentage Devoted Customer Redemptions
disca_devperc = ((disc_a
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_a
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)
discb_devperc = ((disc_b
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_b
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)
discc_devperc = ((disc_c
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_c
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)
discd_devperc = ((disc_d
     .query('cust_level in ("Devoted")')
     .query('hrs_rcvd_redeem >= 0')
     .loc[:, 'cust_level'].count() / 
    disc_d
     .query('cust_level in ("Devoted")')
     .loc[:, 'cust_level'].count()) * 100)

discname = ['Disc A','Disc B','Disc C','Disc D']
disc_rec = [disca_rec, discb_rec, discc_rec, discd_rec]
disc_view = [disca_view, discb_view, discc_view, discd_redcnt]
disc_redcnt = [disca_redcnt, discb_redcnt, discc_redcnt, discd_redcnt]
disc_redperc = [disca_redperc, discb_redperc, discc_redperc, discd_redperc]
disc_redperc_24 = [disca_redperc_24, discb_redperc_24, discc_redperc_24, discd_redperc_24]
disc_rcpt = [disca_rcpt, discb_rcpt, discc_rcpt, discd_rcpt]
disc_3554 = [disca_3554, discb_3554, discc_3554, discd_3554]
disc_5574 = [disca_5574, discb_5574, discc_5574, discd_5574]
disc_stanperc = [disca_stanperc, discb_stanperc, discc_stanperc, discd_stanperc]
disc_loyperc = [disca_loyperc, discb_loyperc, discc_loyperc, discd_loyperc]
disc_devperc = [disca_devperc, discb_devperc, discc_devperc, discd_devperc]


disc_sum_data = ({
    'Campaign': discname,
    'Offers Provided': disc_rec,
    'Offers Viewed': disc_view,
    'Offers Redeemed': disc_redcnt,
    'Average Hours to View': disc_rcpt,
    'Redemption Percent': disc_redperc,
    '24-Hour Redemption Percent': disc_redperc_24})

disc_seg_data = ({
    'Campaign': discname,
    'Percent 35-54 Redeemed': disc_3554,
    'Percent 55-74 Redeemed': disc_5574,
    'Standard Percent Redeemed': disc_stanperc,
    'Loyal Percent Redeemed': disc_loyperc,
    'Devoted Percent Redeemed': disc_devperc})
    
disc_summary = pd.DataFrame(disc_sum_data).round(2)
disc_segmentation = pd.DataFrame(disc_seg_data).round(2)

In [36]:
#Creating for loop to build redemptions per 4 hour span for bogos for first 3 days - DISCOUNTS

rows = []

# using for loop to append values in rows
for i in range(42):
    #appending the values to the row
    if i == 0:
        rows.append([0,0,0,0,0])
    x = ((i+1) * 6) + 1
    rows.append([x-1,
                ((disc_a['hrs_rcvd_redeem'] < x) * 1).sum() / disc_a['customer_id'].notna().sum(),
                ((disc_b['hrs_rcvd_redeem'] < x) * 1).sum() / disc_b['customer_id'].notna().sum(),
                ((disc_c['hrs_rcvd_redeem'] < x) * 1).sum() / disc_c['customer_id'].notna().sum(),
                ((disc_d['hrs_rcvd_redeem'] < x) * 1).sum() / disc_d['customer_id'].notna().sum()])
        
# converting the list into DataFrame
disc_redtimeline = pd.DataFrame(rows, columns=["Time","Discount A","Discount B","Discount C","Discount D"])

In [37]:
#Creating for loop to build redemptions per 4 hour span for bogos for first 5 days - BOGOS

rows = []

# using for loop to append values in rows
for i in range(42):
    #appending the values to the row
    if i == 0:
        rows.append([0,0,0,0,0])
    x = ((i+1) * 6) + 1
    rows.append([x-1,
                ((bogo_a['hrs_rcvd_redeem'] < x) * 1).sum() / bogo_a['customer_id'].notna().sum(),
                ((bogo_b['hrs_rcvd_redeem'] < x) * 1).sum() / bogo_b['customer_id'].notna().sum(),
                ((bogo_c['hrs_rcvd_redeem'] < x) * 1).sum() / bogo_c['customer_id'].notna().sum(),
                ((bogo_d['hrs_rcvd_redeem'] < x) * 1).sum() / bogo_d['customer_id'].notna().sum()])
        
# converting the list into DataFrame
bogo_redtimeline = pd.DataFrame(rows, columns=["Time","Bogo A","Bogo B","Bogo C","Bogo D"])

#Setting indexes to be time elapsed between send and redeem
bogo_redtimeline.set_index(['Time'],inplace=True)
disc_redtimeline.set_index(['Time'],inplace=True)

#Changing format to be float representation of percentage
bogo_redtimeline = (bogo_redtimeline*100).round(2)
disc_redtimeline = (disc_redtimeline*100).round(2)

## Cleaning up tables so we don't have to do it in our dash/plotly sheets

In [38]:
bogo_redtimeline.rename(columns={"Time": "Hours After Send"},inplace=True)
disc_redtimeline.rename(columns={"Time": "Hours After Send"},inplace=True)
cust_profiling.rename(columns=
               {'customer_id':'Customer', 'ttl_spend':'Total Spent', 'avg_spend':'Average Spent Per Visit',
               'visit_cnt':'Visit Count', 'became_member_on':'Became Member On', 'gender':'Gender',
               'age':'Age', 'income':'Income', 'member_age':'Member Age',
               'age_group':'Age Group', 'spend_level':'Spending Level', 'inc_level':'Income Level',
               'member_age_grp':'Membership Age'},
               inplace=True)

#Merging epoch tables to be able to use them more easily in the same chart
info_a['send'] = 'Info A Send'
info_b['send'] = 'Info B Send'

bogo_a_trans['send'] = 'Bogo A Send'
bogo_b_trans['send'] = 'Bogo B Send'
bogo_c_trans['send'] = 'Bogo C Send'
bogo_d_trans['send'] = 'Bogo D Send'

disc_a_trans['send'] = 'Disc A Send'
disc_b_trans['send'] = 'Disc B Send'
disc_c_trans['send'] = 'Disc C Send'
disc_d_trans['send'] = 'Disc D Send'

epochs = pd.concat(
    [info_a, info_b, bogo_a_trans, bogo_b_trans, bogo_c_trans, bogo_d_trans,
    disc_a_trans, disc_b_trans, disc_c_trans, disc_d_trans])

## Final check of all dataframes to verify information looks good prior to export.

In [41]:
 epochs.head(1)


,dt,customer_id,trans,offer_rcvd,offer_viewed,hrdiff,send
0,2024-07-08,b2f1cd155b864803ad8334cdf13c4bd2,19.51,2024-07-15,2024-07-16 06:00:00,-198.0,Info A Send


In [42]:
trans_log.head(1)

,dt,customer_id,trans
12654,2024-07-08,02c083884c7d45b39cc68e1314fec56c,0.83


In [43]:
info_a.head(1)

,dt,customer_id,trans,offer_rcvd,offer_viewed,hrdiff,send
0,2024-07-08,b2f1cd155b864803ad8334cdf13c4bd2,19.51,2024-07-15,2024-07-16 06:00:00,-198.0,Info A Send


In [ ]:
info_b.head(1)

In [44]:
cust_profiling.head(1)

,Customer,Total Spent,Average Spent Per Visit,Visit Count,Became Member On,Gender,Age,Income,Member Age,Age Group,Spending Level,Income Level,Membership Age
0,0009655768c64bdeb2e877511632db8f,128.0,16.0,8,2017-04-21,M,33,72000.0,7.214237,18 to 34,Devoted,Middle,8 years


In [ ]:
disc_redtimeline.head(1)

In [ ]:
bogo_redtimeline.head(1)

In [ ]:
disc_summary

In [ ]:
bogo_summary

In [ ]:
disc_segmentation

In [ ]:
bogo_segmentation

In [ ]:
#Outputs to CSV
epochs.to_csv('data/Promo Transactional Effect.csv', index=False)
disc_summary.to_csv('data/Discount Summary.csv', index=False)
bogo_summary.to_csv('data/Bogo Summary.csv', index=False)
disc_segmentation.to_csv('data/Discount Segmentation.csv', index=False)
bogo_segmentation.to_csv('data/Bogo Segmentation.csv', index=False)
bogo_redtimeline.to_csv('data/Bogo Redemption Timeline.csv')
disc_redtimeline.to_csv('data/Discount Redemption Timeline.csv')
cust_profiling.to_csv('data/Customer Profiling.csv', index=False)